# Sudoku miniproject

Below you will find the most elegant solution of sudoku solver coded by Peter Norvig. It's a good notation and implementation of two (simple) techniques that are enough to solve sudoku in a reasonable time.

You can find the original post here: http://norvig.com/sudoku.html

The best website about sudoku: http://www.sudokudragon.com/sudokutheory.htm

### Sudoku Notation and Preliminary Notions

First we have to agree on some notation. A Sudoku puzzle is a grid of 81 squares; the majority of enthusiasts label the columns 1-9, the rows A-I, and call a collection of nine squares (column, row, or box) a unit and the squares that share a unit the peers. A puzzle leaves some squares blank and fills others with digits, and the whole idea is:
A puzzle is solved if the squares in each unit are filled with a permutation of the digits 1 to 9.
That is, no digit can appear twice in a unit, and every digit must appear once. This implies that each square must have a different value from any of its peers. Here are the names of the squares, a typical puzzle, and the solution to the puzzle:

```
 A1 A2 A3| A4 A5 A6| A7 A8 A9    4 . . |. . . |8 . 5     4 1 7 |3 6 9 |8 2 5 
 B1 B2 B3| B4 B5 B6| B7 B8 B9    . 3 . |. . . |. . .     6 3 2 |1 5 8 |9 4 7
 C1 C2 C3| C4 C5 C6| C7 C8 C9    . . . |7 . . |. . .     9 5 8 |7 2 4 |3 1 6 
---------+---------+---------    ------+------+------    ------+------+------
 D1 D2 D3| D4 D5 D6| D7 D8 D9    . 2 . |. . . |. 6 .     8 2 5 |4 3 7 |1 6 9 
 E1 E2 E3| E4 E5 E6| E7 E8 E9    . . . |. 8 . |4 . .     7 9 1 |5 8 6 |4 3 2 
 F1 F2 F3| F4 F5 F6| F7 F8 F9    . . . |. 1 . |. . .     3 4 6 |9 1 2 |7 5 8 
---------+---------+---------    ------+------+------    ------+------+------
 G1 G2 G3| G4 G5 G6| G7 G8 G9    . . . |6 . 3 |. 7 .     2 8 9 |6 4 3 |5 7 1 
 H1 H2 H3| H4 H5 H6| H7 H8 H9    5 . . |2 . . |. . .     5 7 3 |2 9 1 |6 8 4 
 I1 I2 I3| I4 I5 I6| I7 I8 I9    1 . 4 |. . . |. . .     1 6 4 |8 7 5 |2 9 3 
```

Every square has exactly 3 units and 20 peers. For example, here are the units and peers for the square C2:

```
    A2   |         |                    |         |            A1 A2 A3|         |         
    B2   |         |                    |         |            B1 B2 B3|         |         
    C2   |         |            C1 C2 C3| C4 C5 C6| C7 C8 C9   C1 C2 C3|         |         
---------+---------+---------  ---------+---------+---------  ---------+---------+---------
    D2   |         |                    |         |                    |         |         
    E2   |         |                    |         |                    |         |         
    F2   |         |                    |         |                    |         |         
---------+---------+---------  ---------+---------+---------  ---------+---------+---------
    G2   |         |                    |         |                    |         |         
    H2   |         |                    |         |                    |         |         
    I2   |         |                    |         |                    |         |    
```

In [13]:
#notation

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [a+b for a in A for b in B]

digits   = '123456789'
rows     = 'ABCDEFGHI'
cols     = digits
squares  = cross(rows, cols)
unitlist = ([cross(rows, c) for c in cols] +
            [cross(r, cols) for r in rows] +
            [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')])
units = dict((s, [u for u in unitlist if s in u]) 
             for s in squares)
peers = dict((s, set(sum(units[s],[]))-set([s]))
             for s in squares)
# print(unitlist)
# print(units['A1'])
print(peers)

{'A1': {'D1', 'A8', 'B2', 'A4', 'B3', 'A7', 'I1', 'F1', 'A2', 'A5', 'A9', 'A6', 'E1', 'B1', 'C1', 'A3', 'G1', 'C2', 'H1', 'C3'}, 'A2': {'A8', 'B2', 'A1', 'A4', 'H2', 'B3', 'A7', 'C3', 'G2', 'A5', 'A9', 'E2', 'A6', 'B1', 'C1', 'A3', 'D2', 'C2', 'I2', 'F2'}, 'A3': {'A8', 'I3', 'B2', 'A1', 'A4', 'B3', 'A7', 'G3', 'H3', 'A2', 'A5', 'A9', 'A6', 'D3', 'B1', 'C1', 'F3', 'E3', 'C2', 'C3'}, 'A4': {'E4', 'A8', 'D4', 'G4', 'A1', 'C4', 'A7', 'A2', 'A5', 'A9', 'I4', 'A6', 'F4', 'H4', 'B5', 'C5', 'A3', 'C6', 'B6', 'B4'}, 'A5': {'A8', 'A1', 'A4', 'H5', 'C4', 'A7', 'D5', 'E5', 'G5', 'A2', 'A9', 'A6', 'B5', 'C5', 'A3', 'C6', 'I5', 'F5', 'B6', 'B4'}, 'A6': {'A8', 'A1', 'A4', 'D6', 'F6', 'C4', 'H6', 'A7', 'E6', 'G6', 'A2', 'I6', 'A5', 'A9', 'B5', 'C5', 'A3', 'C6', 'B6', 'B4'}, 'A7': {'A8', 'B8', 'A1', 'E7', 'A4', 'B9', 'H7', 'G7', 'C7', 'I7', 'A2', 'A5', 'A9', 'A6', 'D7', 'B7', 'A3', 'F7', 'C9', 'C8'}, 'A8': {'B8', 'G8', 'A1', 'H8', 'A4', 'B9', 'F8', 'A7', 'C7', 'A2', 'A5', 'D8', 'A9', 'I8', 'A6', 'E8', 

### Sudoku grid
Now that we have squares, units, and peers, the next step is to define the Sudoku playing grid. Actually we need two representations: First, a textual format used to specify the initial state of a puzzle; we will reserve the name grid for this. Second, an internal representation of any state of a puzzle, partially solved or complete; this we will call a values collection because it will give all the remaining possible values for each square. For the textual format (grid) we'll allow a string of characters with 1-9 indicating a digit, and a 0 or period specifying an empty square. All other characters are ignored (including spaces, newlines, dashes, and bars). So each of the following three grid strings represent the same puzzle:



In [14]:
# parser
def grid_values(grid):
    "Convert grid into a dict of {square: char} with '0' or '.' for empties."
    chars = [c for c in grid if c in digits or c in '0.']
    assert len(chars) == 81
    return dict(zip(squares, chars))

def display(values):
    "Display these values as a 2-D grid."
    width = 1+max(len(values[s]) for s in squares)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '') for c in cols))
        if r in 'CF': print(line)
    print()

# sample sudoku
grid1 = '003020600900305001001806400008102900700000008006708200002609500800203009005010300'
grid2 = "4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......"
display(grid_values(grid2))
display(grid_values(grid1))
print(grid_values(grid2))

4 . . |. . . |8 . 5 
. 3 . |. . . |. . . 
. . . |7 . . |. . . 
------+------+------
. 2 . |. . . |. 6 . 
. . . |. 8 . |4 . . 
. . . |. 1 . |. . . 
------+------+------
. . . |6 . 3 |. 7 . 
5 . . |2 . . |. . . 
1 . 4 |. . . |. . . 

0 0 3 |0 2 0 |6 0 0 
9 0 0 |3 0 5 |0 0 1 
0 0 1 |8 0 6 |4 0 0 
------+------+------
0 0 8 |1 0 2 |9 0 0 
7 0 0 |0 0 0 |0 0 8 
0 0 6 |7 0 8 |2 0 0 
------+------+------
0 0 2 |6 0 9 |5 0 0 
8 0 0 |2 0 3 |0 0 9 
0 0 5 |0 1 0 |3 0 0 

{'A1': '4', 'A2': '.', 'A3': '.', 'A4': '.', 'A5': '.', 'A6': '.', 'A7': '8', 'A8': '.', 'A9': '5', 'B1': '.', 'B2': '3', 'B3': '.', 'B4': '.', 'B5': '.', 'B6': '.', 'B7': '.', 'B8': '.', 'B9': '.', 'C1': '.', 'C2': '.', 'C3': '.', 'C4': '7', 'C5': '.', 'C6': '.', 'C7': '.', 'C8': '.', 'C9': '.', 'D1': '.', 'D2': '2', 'D3': '.', 'D4': '.', 'D5': '.', 'D6': '.', 'D7': '.', 'D8': '6', 'D9': '.', 'E1': '.', 'E2': '.', 'E3': '.', 'E4': '.', 'E5': '8', 'E6': '.', 'E7': '4', 'E8': '.', 'E9': '.', 'F1': '.', 'F2': '.', 'F3': '.', 'F4': '

### Parser
Now for values. One might think that a 9 x 9 array would be the obvious data structure. But squares have names like 'A1', not (0,0). Therefore, values will be a dict with squares as keys. The value of each key will be the possible digits for that square: a single digit if it was given as part of the puzzle definition or if we have figured out what it must be, and a collection of several digits if we are still uncertain. This collection of digits could be represented by a Python set or list, but I chose instead to use a string of digits (we'll see why later). So a grid where A1 is 7 and C7 is empty would be represented as {'A1': '7', 'C7': '123456789', ...}.

In [22]:
def parse_to_raw_dict(grid):
    values = grid_values(grid)
    for k in values.keys():
        if values[k] not in digits:
            values[k] = digits
    return values
            
def parse_grid(grid):
    """Convert grid to a dict of possible values, {square: digits}, or
    return False if a contradiction is detected."""
    ## To start, every square can be any digit; then assign values from the grid.
    values = dict((s, digits) for s in squares)
    for s,d in grid_values(grid).items():
        if d in digits and not assign(values, s, d):
            return False ## (Fail if we can't assign d to square s.)
    return values

grid2 = "4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......"
sudoku_values = (grid_values(grid2))
                 
print(sudoku_values)

{'A1': '4', 'A2': '.', 'A3': '.', 'A4': '.', 'A5': '.', 'A6': '.', 'A7': '8', 'A8': '.', 'A9': '5', 'B1': '.', 'B2': '3', 'B3': '.', 'B4': '.', 'B5': '.', 'B6': '.', 'B7': '.', 'B8': '.', 'B9': '.', 'C1': '.', 'C2': '.', 'C3': '.', 'C4': '7', 'C5': '.', 'C6': '.', 'C7': '.', 'C8': '.', 'C9': '.', 'D1': '.', 'D2': '2', 'D3': '.', 'D4': '.', 'D5': '.', 'D6': '.', 'D7': '.', 'D8': '6', 'D9': '.', 'E1': '.', 'E2': '.', 'E3': '.', 'E4': '.', 'E5': '8', 'E6': '.', 'E7': '4', 'E8': '.', 'E9': '.', 'F1': '.', 'F2': '.', 'F3': '.', 'F4': '.', 'F5': '1', 'F6': '.', 'F7': '.', 'F8': '.', 'F9': '.', 'G1': '.', 'G2': '.', 'G3': '.', 'G4': '6', 'G5': '.', 'G6': '3', 'G7': '.', 'G8': '7', 'G9': '.', 'H1': '5', 'H2': '.', 'H3': '.', 'H4': '2', 'H5': '.', 'H6': '.', 'H7': '.', 'H8': '.', 'H9': '.', 'I1': '1', 'I2': '.', 'I3': '4', 'I4': '.', 'I5': '.', 'I6': '.', 'I7': '.', 'I8': '.', 'I9': '.'}


# Constraint Propagation - original code
The function parse_grid calls assign(values, s, d). We could implement this as values[s] = d, but we can do more than just that. Those with experience solving Sudoku puzzles know that there are two important strategies that we can use to make progress towards filling in all the squares:

(1) If a square has only one possible value, then eliminate that value from the square's peers. 

(2) If a unit has only one possible place for a value, then put the value there.

As an example of strategy (1) if we assign 7 to A1, yielding {'A1': '7', 'A2':'123456789', ...}, we see that A1 has only one value, and thus the 7 can be removed from its peer A2 (and all other peers), giving us {'A1': '7', 'A2': '12345689', ...}. As an example of strategy (2), if it turns out that none of A3 through A9 has a 3 as a possible value, then the 3 must belong in A2, and we can update to {'A1': '7', 'A2':'3', ...}. These updates to A2 may in turn cause further updates to its peers, and the peers of those peers, and so on. This process is called constraint propagation.

The function assign(values, s, d) will return the updated values (including the updates from constraint propagation), but if there is a contradiction--if the assignment cannot be made consistently--then assign returns False. For example, if a grid starts with the digits '77...' then when we try to assign the 7 to A2, assign would notice that 7 is not a possibility for A2, because it was eliminated by the peer, A1.

It turns out that the fundamental operation is not assigning a value, but rather eliminating one of the possible values for a square, which we implement with eliminate(values, s, d). Once we have eliminate, then assign(values, s, d) can be defined as "eliminate all the values from s except d".

In [23]:
def assign(values, s, d):
    
    """Eliminate all the other values (except d) from values[s] and propagate.
    Return values, except return False if a contradiction is detected."""
    other_values = values[s].replace(d, '')
    if all(eliminate(values, s, d2) for d2 in other_values):
        return values
    else:
        return False

def eliminate(values, s, d):
    """Eliminate d from values[s]; propagate when values or places <= 2.
    Return values, except return False if a contradiction is detected."""
    print(values)
    if d not in values[s]:
        return values ## Already eliminated
    values[s] = values[s].replace(d,'')
    ## (1) If a square s is reduced to one value d2, then eliminate d2 from the peers.
    if len(values[s]) == 0:
        return False ## Contradiction: removed last value
    elif len(values[s]) == 1:
        d2 = values[s]
        if not all(eliminate(values, s2, d2) for s2 in peers[s]):
            return False
    ## (2) If a unit u is reduced to only one place for a value d, then put it there.
    for u in units[s]:
        dplaces = [s for s in u if d in values[s]]
        if len(dplaces) == 0:
            return False ## Contradiction: no place for this value
        elif len(dplaces) == 1:
            # d can only be in one place in unit; assign it there
            if not assign(values, dplaces[0], d):
                return False
    return values

In [24]:
import time

### Test run

In [25]:
grid2 = "4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......"
start= time.clock()
sample_grid = parse_to_raw_dict(grid1)
sudoku_values = (parse_grid(grid1))
print(time.clock()-start)
display(sudoku_values)
display(sample_grid)

{'A1': '123456789', 'A2': '123456789', 'A3': '123456789', 'A4': '123456789', 'A5': '123456789', 'A6': '123456789', 'A7': '123456789', 'A8': '123456789', 'A9': '123456789', 'B1': '123456789', 'B2': '123456789', 'B3': '123456789', 'B4': '123456789', 'B5': '123456789', 'B6': '123456789', 'B7': '123456789', 'B8': '123456789', 'B9': '123456789', 'C1': '123456789', 'C2': '123456789', 'C3': '123456789', 'C4': '123456789', 'C5': '123456789', 'C6': '123456789', 'C7': '123456789', 'C8': '123456789', 'C9': '123456789', 'D1': '123456789', 'D2': '123456789', 'D3': '123456789', 'D4': '123456789', 'D5': '123456789', 'D6': '123456789', 'D7': '123456789', 'D8': '123456789', 'D9': '123456789', 'E1': '123456789', 'E2': '123456789', 'E3': '123456789', 'E4': '123456789', 'E5': '123456789', 'E6': '123456789', 'E7': '123456789', 'E8': '123456789', 'E9': '123456789', 'F1': '123456789', 'F2': '123456789', 'F3': '123456789', 'F4': '123456789', 'F5': '123456789', 'F6': '123456789', 'F7': '123456789', 'F8': '1234

# Constraint Propagation - code refactoring

Try to decompose* eliminate function into smaller one.
* Decomposition is a process by which you can break down one complex function into multiple smaller functions. By doing this, you can solve for functions in shorter, easier-to-understand pieces.

In [28]:
# todo
import copy

def eliminate(values):
    """
    Iterate through all squares and every time 
       if there is a square with one value, 
       then eliminate this value from the peers

    input: sudoku in dictionary form
    output: resulting sudoku in dictionary form
    """
    for key in values.keys():
        if len(values[key]) == 0:
            return False
        elif len(values[key]) == 1:            
            for peer in peers[key]:
                values[peer] = values[peer].replace(values[key],'')
    return values


def only_choice(values):
    """
    Iterate through all squares and every time
        if there is a square with a value that only fits in one square, 
        assign the value to this square

    input: sudoku in dictionary form
    output: resulting sudoku in dictionary form
    """
    digits = '123456789'
    rows = 'ABCDEFGHI'
    
    for key in values.keys():
        if len(values[key]) == 0:
            return False
        elif len(values[key]) != 1:
            for numbers in values[key]:
                letter = key[0]
                number = key[1]

                rows_ind = rows.find(letter)
                rows_ind = rows_ind - (rows_ind%3)
                digits_ind = digits.find(number)
                digits_ind = digits_ind - (digits_ind%3)
                unit = cross(rows[rows_ind : rows_ind+3], digits[digits_ind : digits_ind+3])

                for unit in units[key]:
                    digits_not_found = digits[:]
                    for sqr in unit:
                        if sqr != key:
                            digits_not_found = digits_not_found.translate(str.maketrans('', '', values[sqr]))
                    if len(digits_not_found)==1 and digits_not_found in values[key]:
                        values[key] = digits_not_found
                        break
    return values

def is_solved(values):
    """
    Check if sudoku is solved
    
    param: sudoku in dictionary form
    return: boolean is_solved
    """
    for value in values:
        if len(values[value]) != 1:
            return False;
    return True


def reduce_puzzle(values):
    """
    Solve sudoku using eliminate() and only_choice()
    
    input: sudoku in dictionary form
    output: resulting sudoku in dictionary form
    """
    while not is_solved(values):
        previous_values = copy.deepcopy(values)
        values = eliminate(values)
        values = only_choice(values)
        if(previous_values == values):
            return values
    return values

In [29]:
display(reduce_puzzle(sample_grid))

4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 



# Naked twins
http://www.sudokudragon.com/tutorialnakedtwins.htm


In [41]:
def naked_twins(values):
    """
    eliminate values using the naked twins strategy
    
    input: A sudoku in dictionary form.
    output: The resulting sudoku in dictionary form.
    """
    for x in values.keys():
        if len(values[x]) == 0:
            return False
        elif len(values[x]) != 1:
            for unit in units[x]:
                # in a single unit...
                the_same_fields = list()
                for sqr in unit:
                    if values[x] == values[sqr]:
                        the_same_fields.append(sqr)
                if len(values[x]) == len(the_same_fields):
                    wipe_naked_permutations = str.maketrans('', '', values[x])
                    for sqr in unit:
                        if sqr not in the_same_fields and len(values[sqr]) != 1:
                            values[sqr] = values[sqr].translate(wipe_naked_permutations)
    return values
    
def hidden_twins(values):
    """
    Eliminate values using the hidden twins strategy
    
    input: A sudoku in dictionary form.
    output: The resulting sudoku in dictionary form.
    """
    for x in values.keys():
        if len(values[x]) == 0:
            return False
        elif len(values[x]) != 1: # when the square is not solved...
            for unit in units[x]:
                possible_vals = {}
                for i in values[x]:
                    possible_vals[i] = list()
                for sqr in unit:
                    for possibility in values[sqr]:
                        if possibility in possible_vals:
                            possible_vals[possibility].append(sqr)
                for possibility1 in possible_vals.keys():
                    identical_count = 0
                    identical_numbers = ''
                    for possibility2 in possible_vals.keys():
                        if possible_vals[possibility1] == possible_vals[possibility2]:
                            identical_numbers = identical_numbers + possibility2
                            identical_count = identical_count + 1
                    if identical_count == len(possible_vals[possibility1]):
                        for sqr in possible_vals[possibility1]:
                            if len(identical_numbers) < len(values[sqr]): #if it really reduces the problem...
                                values[sqr] = ''.join(sorted(identical_numbers))
                        # we want to check if n values from x square are also in exactly n other squares
    return values

def reduce_with_twins(values):
    """
    Solve sudoku using eliminate() and only_choice()
    
    input: sudoku in dictionary form
    output: resulting sudoku in dictionary form
    """
    while not is_solved(values):
        previous_values = copy.deepcopy(values)
        values = eliminate(values)
        values = only_choice(values)
        values = naked_twins(values)
        values = hidden_twins(values)
        if(previous_values == values): #if there is no progress, just give up
            return values
    return values

In [60]:
'''
Brute-force, backtracking Sudoku solver in about fifteen lines.

Works on Python 2.6 and Python 3.
'''
def brute_solve(s):
    ''' 
    Solve a Sudoku:

    - Accepts s, a sequence of 81 integers from 0 to 9 in row of
    column order, zeros indicating the cells to fill.

    - Returns the first found solution as a sequence of 81 integers in
    the 1 to 9 interval (same row or column order than input), or None
    if no solution exists.
    '''
    try:
        i  = s.index(0)
    except ValueError: 
        # No empty cell left: solution found
        return s

    c = [s[j] for j in range(81)
         if not ((i-j)%9 * (i//9^j//9) * (i//27^j//27 | (i%9//3^j%9//3)))]

    for v in range(1, 10):
        if v not in c:
            r = solve(s[:i]+[v]+s[i+1:])
            if r is not None:
                return r

#-------------------------------------------------------------------------------
# Let's test it!
#
if __name__ == '__main__':
    class Sudoku(list):
        '''Sudokus with nicer IO'''
        def __init__(self, content):
            list.__init__(self, [int(i) for i in content.split()] 
                          if isinstance(content, str) else content)
        def __str__(self):
            return '\n'.join(
                ' '.join([(str(j) if j != 0 else '-') 
                          for j in self[i*9:(i+1)*9]]) for i in range(9))

    problem = Sudoku('''
        5 3 0 0 7 0 0 0 0
        6 0 0 1 9 5 0 0 0
        0 9 8 0 0 0 0 6 0
        8 0 0 0 6 0 0 0 3
        4 0 0 8 0 3 0 0 1
        7 0 0 0 2 0 0 0 6
        0 6 0 0 0 0 2 8 0
        0 0 0 4 1 9 0 0 5
        0 0 0 0 8 0 0 7 9
        ''')

    solution = Sudoku('''
        5 3 4 6 7 8 9 1 2
        6 7 2 1 9 5 3 4 8
        1 9 8 3 4 2 5 6 7
        8 5 9 7 6 1 4 2 3
        4 2 6 8 5 3 7 9 1
        7 1 3 9 2 4 8 5 6
        9 6 1 5 3 7 2 8 4
        2 8 7 4 1 9 6 3 5
        3 4 5 2 8 6 1 7 9
        ''')
    start= time.clock()
    result = Sudoku(solve(problem))
    print(time.clock()-start)

    print('==== Problem ====\n{0}\n\n=== Solution ====\n{1}'.format(
            problem, result))
    
    assert(result == solution)

0.14247199999999793
==== Problem ====
5 3 - - 7 - - - -
6 - - 1 9 5 - - -
- 9 8 - - - - 6 -
8 - - - 6 - - - 3
4 - - 8 - 3 - - 1
7 - - - 2 - - - 6
- 6 - - - - 2 8 -
- - - 4 1 9 - - 5
- - - - 8 - - 7 9

=== Solution ====
5 3 4 6 7 8 9 1 2
6 7 2 1 9 5 3 4 8
1 9 8 3 4 2 5 6 7
8 5 9 7 6 1 4 2 3
4 2 6 8 5 3 7 9 1
7 1 3 9 2 4 8 5 6
9 6 1 5 3 7 2 8 4
2 8 7 4 1 9 6 3 5
3 4 5 2 8 6 1 7 9


In [59]:
from functools import reduce

def solve_sudokus(filename):

    file = open(filename)
    count_all = 0
    solved_simple = 0
    solved_twins = 0
    simple_times = list([0])
    twins_times= list([0])

    for sudoku in file:
        count_all = count_all + 1
        puzzle_for_twins = parse_to_raw_dict(sudoku)
        puzzle_for_simple = copy.deepcopy(puzzle_for_twins)
        start = time.clock()
        simple_solution = reduce_puzzle(puzzle_for_simple)
        simple_done = time.clock()
        twins_solution = reduce_with_twins(puzzle_for_twins)
        twins_done = time.clock()

        if is_solved(simple_solution):
            solved_simple = solved_simple + 1
        simple_times.append(simple_done-start)

        if is_solved(twins_solution):
            solved_twins = solved_twins + 1
        twins_times.append(twins_done-simple_done)

    
    sp_time = reduce((lambda x, y: x + y), simple_times)
    tw_time = reduce((lambda x, y: x + y), twins_times)

    print(filename + ' statistics:\n')
    print("{}/{} of all puzzles successfully solved with simple method".format(solved_simple, count_all))
    print("Simple method took %.3fs" % sp_time, "overall.\n")

    print("{}/{} of all puzzles successfully solved with twins method".format(solved_twins, count_all))
    print("Twins took %.3fs" % tw_time, "overall.")

    print('{}'.format('-'*100),"\n\n")


solve_sudokus('files/easy50.txt')
solve_sudokus('files/top95.txt')
solve_sudokus('files/hardest.txt')

files/easy50.txt statistics:

40/50 of all puzzles successfully solved with simple method
Simple method took 0.474s overall.

46/50 of all puzzles successfully solved with twins method
Twins took 0.447s overall.
---------------------------------------------------------------------------------------------------- 


files/top95.txt statistics:

0/95 of all puzzles successfully solved with simple method
Simple method took 1.475s overall.

12/95 of all puzzles successfully solved with twins method
Twins took 1.926s overall.
---------------------------------------------------------------------------------------------------- 


files/hardest.txt statistics:

1/11 of all puzzles successfully solved with simple method
Simple method took 0.136s overall.

2/11 of all puzzles successfully solved with twins method
Twins took 0.154s overall.
---------------------------------------------------------------------------------------------------- 




# Summary
Both simple method and twins method produce good satisfactory results when solving easy sudokus. Only twins method can solve some of the top95 sudokus. Overall both methods are not satisfactory when it comes to harder sudokus and improvements must be made in them to actually approach harder sudokus.